In [36]:
import pandas as pd
import csv
from matplotlib import pyplot as plt
import os
import numpy as np
from scipy import stats


In [37]:
folder_list=list()
current_folder = os.getcwd()
for item in os.listdir(current_folder):
    if len(item) <= 3:
        folder_list.append(item)
folder_list.sort(key=lambda x:int(x.replace("G","")))

In [38]:
csv_list=list()
for folder in folder_list:
    folder_path=os.path.join(current_folder,folder)
    csv_list.append(os.listdir(folder_path))
gain_list=list()
for folder in folder_list:
    gain_list.append(int(folder.replace("G","")))

In [39]:
sum=0
for csv_file in csv_list:
    sum+=len(csv_file)
print(sum)

54


In [40]:
V_mm = list()
V_adc = list()
G = list()
G_LSB = list()

In [41]:
for i in range(len(csv_list)):
    for csv_file in csv_list[i]:

        G.append(gain_list[i])
        G_LSB.append(4096/2**15/gain_list[i])
        splitted_I = csv_file.replace(".csv", "").split(" ")
        V_mm.append(float(splitted_I[0]) +
                float(splitted_I[1])/(10**len(splitted_I[1])))

        folder_path=os.path.join(current_folder,folder_list[i])
        with open(os.path.join(folder_path,csv_file), "r") as f:
            csv_reader = csv.reader(f, delimiter=',')
            V_adc_temp = list()
            for row in csv_reader:
                # cek kalo list kosong atau engga
                if row:
                    V_adc_temp.append(float(row[0]))
        V_adc.append(np.average(V_adc_temp))

In [42]:
Vdata = pd.DataFrame([V_mm, V_adc, G, G_LSB]).T
Vdata.columns = ['V multimeter', 'V ADC', 'G', 'G LSB']
Vdata=Vdata.sort_values(["G","V multimeter"]).reset_index(drop=True)
#Vdata = Vdata.sort_values(['G','V ADC'])
#Vdata['V ADC'] = Vdata['V ADC']*3/6.2/1000

In [43]:
Vdata['V ADC (mV)'] = Vdata['V ADC'].multiply(Vdata['G LSB'], axis="index")

\begin{equation*}
  V_{adc}=I_{in}R_{shunt}\frac{1}{3}A_v
\end{equation*}
\begin{equation*}
    I_{in}=\frac{3V_{adc}}{R_{shunt}A_v}
\end{equation*}


In [44]:
Slope_I = list()
Inter_I = list()
for gain in gain_list:
    reg_result= stats.linregress(Vdata['V ADC (mV)'].loc[Vdata['G']==gain], Vdata['V multimeter'].loc[Vdata['G']==gain])
    Slope_I.append(reg_result.slope)
    Inter_I.append(reg_result.intercept)


In [45]:
Slope_arr= list()
Inter_arr= list()
#reg_pred = list()
for i in range(len(csv_list)):
    for j in csv_list[i]:
        Slope_arr.append(Slope_I[i])
        Inter_arr.append(Inter_I[i])

In [46]:
Vdata['V pred']=Vdata['V ADC (mV)'].multiply(Slope_arr)+Inter_arr

In [47]:
E_arr=list()
Diff_arr=list()
for i in range(len(Vdata)):
    E_arr.append(abs(Vdata['V pred'][i]-Vdata['V multimeter'][i])/Vdata['V multimeter'][i]*100)
    Diff_arr.append(abs(Vdata['V pred'][i]-Vdata['V multimeter'][i]))
Vdata['Error']=E_arr
Vdata['Diff']=Diff_arr

In [48]:
for i in gain_list:
    print(np.average(Vdata["Error"].loc[Vdata['G']==i]))

0.24956538060559091
0.31169541287164454
0.19210243396807833
0.06253280565341186
4.401352549222193


In [67]:
gain_list

[1, 2, 4, 8, 16]

In [66]:
gain_LSB_list

[0.125, 0.0625, 0.03125, 0.015625, 0.0078125]

In [65]:
Slope_I

[0.020780802192451212,
 0.02084110018034695,
 0.021027857135915068,
 0.021044122249240838,
 0.021282329552675307]

In [68]:
32768*0.125*Slope_I[0]+Inter_I[0]

85.27974783982003

In [61]:
gain_LSB_list

[0.125, 0.0625, 0.03125, 0.015625, 0.0078125]

In [62]:
Slope_I

[0.020780802192451212,
 0.02084110018034695,
 0.021027857135915068,
 0.021044122249240838,
 0.021282329552675307]

In [64]:
np.array(Inter_I)

array([0.16158206, 0.10947509, 0.06196756, 0.0535543 , 0.01093328])

In [59]:
gain_LSB_list=list()
for gain in gain_list:
    gain_LSB_list.append(4096/2**15/gain)

In [52]:
Vdata['V ADC'].iloc[0]*Vdata['G LSB'].iloc[0]*Slope_I[0]+Inter_I[0]

,V multimeter,V ADC,G,G LSB,V ADC (mV),V pred,Error,Diff
0,6.470,2458.7,1.0,0.125000,307.337500,6.548302,1.210230,0.078302
1,10.510,4004.0,1.0,0.125000,500.500000,10.562374,0.498321,0.052374
2,15.750,6003.9,1.0,0.125000,750.487500,15.757314,0.046440,0.007314
3,20.680,7897.5,1.0,0.125000,987.187500,20.676130,0.018713,0.003870
4,25.770,9830.8,1.0,0.125000,1228.850000,25.698071,0.279120,0.071929
5,31.470,11979.0,1.0,0.125000,1497.375000,31.278236,0.609356,0.191764
6,35.400,13525.2,1.0,0.125000,1690.650000,35.294645,0.297612,0.105355
7,40.240,15453.6,1.0,0.125000,1931.700000,40.303858,0.158692,0.063858
8,45.470,17460.7,1.0,0.125000,2182.587500,45.517501,0.104467,0.047501
9,50.560,19424.3,1.0,0.125000,2428.037500,50.618149,0.115010,0.058149


In [49]:
import plotly as ply

In [50]:
fig = ply.graph_objects.Figure()
# Create and style traces
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 16.0], y=Vdata["Error"].loc[Vdata['G'] == 16.0],name="16x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 8.0], y=Vdata["Error"].loc[Vdata['G'] == 8.0],name="8x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 4.0], y=Vdata["Error"].loc[Vdata['G'] == 4.0],name="4x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 2.0], y=Vdata["Error"].loc[Vdata['G'] == 2.0],name="2x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 1.0], y=Vdata["Error"].loc[Vdata['G'] == 1.0],name="1x"))
fig.update_layout(title='Error Pengukuran arus pada setiap gain',
                  xaxis_title='Arus (mA)',
                  yaxis_title='Error (%)')


In [51]:
fig = ply.graph_objects.Figure()
# Create and style traces
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 16.0], y=Vdata["Diff"].loc[Vdata['G'] == 16.0],name="16x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 8.0], y=Vdata["Diff"].loc[Vdata['G'] == 8.0],name="8x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 4.0], y=Vdata["Diff"].loc[Vdata['G'] == 4.0],name="4x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 2.0], y=Vdata["Diff"].loc[Vdata['G'] == 2.0],name="2x"))
fig.add_trace(ply.graph_objects.Scatter(
    x=Vdata["V multimeter"].loc[Vdata['G'] == 1.0], y=Vdata["Diff"].loc[Vdata['G'] == 1.0],name="1x"))
fig.update_layout(title='Error Pengukuran arus pada setiap gain',
                  xaxis_title='Arus (mA)',
                  yaxis_title='Error (%)')
